In [55]:
default_headers = {
          'Content-Type': 'application/json',
          'Access-Control-Allow-Origin': '*'
        }

In [105]:
from IPython.display import display_javascript, display_html, display
import uuid

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 150px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [106]:
import requests, json

In [107]:
engine = "http://docker.for.mac.localhost:8181"

In [108]:
e = requests.get(engine + "/jasper").json()

In [109]:
RenderJSON(e)

In [174]:
l = requests.get(engine + "/jasper/streams").json()

In [175]:
RenderJSON(l)

In [176]:
qs = requests.get(engine + "/jasper/queries").json()

In [177]:
RenderJSON(qs)

In [270]:
body = {
    'id':'q1',
    'tbox': 'file:///Users/riccardo/Desktop/colors.owl',
    'body': 'CONSTRUCT {?s ?p ?o} FROM NAMED WINDOW <w1> ON <http://localhost:1255/red> [RANGE PT10S STEP PT10S] WHERE { WINDOW ?w { ?s ?p ?o } }'
}
    
data = json.dumps(body)

RenderJSON(body)

In [271]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers).json()

In [173]:
RenderJSON(q)

In [272]:
os = requests.get(engine + "/jasper/observers/q2").json()

In [273]:
RenderJSON(os)

In [274]:
data=json.dumps({ 
    'stream': engine + "/jasper/queries/q1",
    'protocol': "WEBSOCKET" 
})

In [275]:
o = requests.post(engine + "/jasper/observers/q1", data = json.dumps({'protocol':"HTTP", "retention":5}) , headers=default_headers).json()

In [276]:
os = requests.get(engine + "/jasper/observers/q1").json()

In [277]:
RenderJSON(o)

In [278]:
RenderJSON(os)

In [280]:
o = requests.post(engine + "/jasper/observers/q1", data = json.dumps({'protocol':"WEBSOCKET"}) , headers=default_headers).json()

In [281]:
RenderJSON(o)

In [287]:
body = {
    'id':'q2',
    'tbox': 'file:///Users/riccardo/Desktop/colors.owl',
    'body': 'CONSTRUCT {?s ?p ?o} FROM NAMED WINDOW <w1> ON <http://localhost:8181/jasper/streams/q1> [RANGE PT10S STEP PT10S] WHERE { WINDOW ?w { ?s ?p ?o } }'
}
    
data = json.dumps(body)

RenderJSON(body)


In [288]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [289]:
q.text

'Not found: http://localhost:8181/jasper/streams/q9'

In [286]:
RenderJSON(q.json())

In [190]:
from rdflib import Graph, plugin
from rdflib.serializer import Serializer

In [292]:
a = requests.get("http://docker.for.mac.localhost:8182/jasper/streams/q1/observers/513111409").json()

In [293]:
RenderJSON(a)

In [239]:
g = Graph().parse(data=a, format='json-ld')

In [240]:
g

<Graph identifier=N20d79d4182b342cfb038f3b88df20959 (<class 'rdflib.graph.Graph'>)>

In [241]:
qres = g.query('''PREFIX vsd: <http://w3id.org/rsp/vocals-sd#>
PREFIX vocals: <http://w3id.org/rsp/vocals#>
PREFIX dcat: <http://www.w3.org/ns/dcat#>

SELECT *
WHERE {

    ?stream a vocals:RDFStream ;
              vocals:hasEndpoint ?endpoint .

    ?endpoint dcat:accessURL ?url ;
              dcat:format ?format ;
              vsd:publishedBy ?service .



}''')

for row in qres:
    print(row)

(rdflib.term.URIRef('http://localhost:8181/jasper/streams/q2'), rdflib.term.Literal('frmt:JSON-LD'), rdflib.term.BNode('N458a702826994fa8b35d0b45b4e772da'), rdflib.term.Literal('ws://localhost:8183/jasper/streams/q2/observers/2107142250', datatype=rdflib.term.URIRef('xsd:string')), rdflib.term.Literal('http://localhost:8181/jasper'))
